In [279]:
#establishing environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat, savemat

In [280]:
split_1 = loadmat('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/matlab_import/split_1.mat')

In [281]:
values_gdf0 = split_1['values_all'][0,0] #30721 x 146
values_gdf1 = split_1['values_all'][0,1]
values_gdf2 = split_1['values_all'][0,2]
values_gdf3 = split_1['values_all'][0,3]
values_gdf4 = split_1['values_all'][0,4]
values_gdf5 = split_1['values_all'][0,5]

seq_0 = split_1['seqs_all'][0,0] #[0,0][:,0] - gives you first seq in gdf 0, channel info, [0,0][:,1] - gives you spike index
seq_1 = split_1['seqs_all'][0,1]
seq_2 = split_1['seqs_all'][0,2]
seq_3 = split_1['seqs_all'][0,3]
seq_4 = split_1['seqs_all'][0,4]
seq_5 = split_1['seqs_all'][0,5]

ch_labels0 = split_1['ch_labels_all'][0,0]
fs = 512;

In [282]:
np.shape(split_1['values_all'])

(1, 6)

In [283]:
#NOTES:
#LOOK INTO MAYBE REMOVING NAN'S BUT THAT WOULD SHIFT THE CHLABELS. 
#PLOT CODE IN MATLAB DOESN'T CHANGE THE NAN'S IT JUST DOESN'T OFFSET THEM. 

In [284]:
print(np.shape(values_gdf0))

(30721, 146)


In [285]:
#function to find the mean_max in a sequence split(gdf)
def mean_max(sequence_split, values):
    seq_0_concat = np.concatenate(sequence_split)
    seq_0_concat = np.concatenate(seq_0_concat) #double concatenate so that everything is in a array corresponding - basically turns into GDF
    ch_uniq = np.unique(seq_0_concat[:,0]) #finds the unique channels in each run_time
    #finds the mean of the max of each channel's spike index.
    meanmax_per_ch = [];
    for ch in ch_uniq:
        x = np.where(seq_0_concat[:,0] == ch)[0] #index where all spikes are per channel per run_time
        max_in_ch = []
        for i in x:
            val = values[seq_0_concat[i,1]-20:seq_0_concat[i,1]+20, ch] #finding value at the spike and channel from x
            val_max = np.max(np.abs(val))
            max_in_ch.append(val_max)
        meanmax_per_ch.append(np.mean(max_in_ch))
    
    return meanmax_per_ch, ch_uniq

In [286]:
mean_max_seq_0 = mean_max(seq_0,values_gdf0)
print(mean_max_seq0)

[410.88361650850544, 154.90517452992887, 147.91623852400187, 244.34705052539755, 112.40112536283877, 204.30030075474554, 161.313233439993, 175.89055598872588, 144.1146710268245, 119.32733383452903, 150.72051541032263, 121.901648108972, 204.9564291929405, 102.78531202685559, 114.01200742988821, 105.87021301179789, 189.4097833904775, 630.8602163350512, 256.3494837394883, 173.0920319224909, 479.64651940249956, 342.6513499202294, 283.0246348092994, 146.08609423136363, 176.0875493565139, 111.23254533067617, 111.769606729618, 109.4634079363243, 486.3134982867389, 125.27184558249105, 179.52362687598782, 177.6891955234385, 192.1357808642966, 184.60736191027917, 141.03070538526111, 187.79121630708485, 185.47759233070164, 117.53363841529887, 69.73865037113053, 228.06500594548126, 215.27187980890648, 192.0685250894378, 287.6640947477094, 119.48713723970621, 84.39192134103361, 53.966891945371685, 284.2849983439468, 232.8002162404095, 182.77680294804352, 49.45968849123526, 206.65107630360475, 80.18

In [287]:
#creates a concatenation of ALL gdf's mean absolute max value in every channel
def ALL_mean_max(split_1): #input would be the complete matrix assuming 'values_all' and 'seqs_all' are the base names
    mean_max_ALL = []
    ch_uniq_ALL = []
    
    for I in range(len(split_1['values_all'][0])):
        values_gdf_I = split_1['values_all'][0,I]
        seq_I = split_1['seqs_all'][0,I]
        mean_max_seq_I, ch_uniq_I = mean_max(seq_I,values_gdf_I)
        mean_max_ALL.append(mean_max_seq_I)
        ch_uniq_ALL.append(ch_uniq_I)
    
    ch_uniq_AL_C = np.concatenate(ch_uniq_ALL)
    mean_max_ALL_C = np.concatenate(mean_max_ALL)
    
    ALL_mean_max_with_ch = [ch_uniq_AL_C,mean_max_ALL_C]
    return ALL_mean_max_with_ch

In [288]:
all_mean_max = ALL_mean_max(split_1)
np.shape(all_mean_max)

(2, 492)

In [289]:
#creates a compilation of every channel in all gdfs and there respective means of max absolute peak values 
def meanofmeanmax_per_ch(split_1): #input is the complete split file.
    all_mean_max = ALL_mean_max(split_1) #uses ALL_mean_max function to get you a complete list of concatenated mean max's
    
    #code to get means of means per channel.
    all_mean_max = np.transpose(all_mean_max)
    ch_uniq = np.unique(all_mean_max[:,0]) #finds the unique channels in concatenated list
    means = []
    for ch in ch_uniq:
        x = np.where(all_mean_max[:,0] == ch)[0] #index where all spikes are per channel per run_time
        means.append(np.mean(all_mean_max[x,1]))
    meanofmeanmax = [ch_uniq,means]    
    return np.transpose(meanofmeanmax) # ['channel','mean max']

In [290]:
meanmax_ch_ALL = meanofmeanmax_per_ch(split_1)

In [293]:
np.shape(meanmax_ch_ALL)

(107, 2)